# Construindo banco de dados para treinamento dos modelos e uso das aplicações

- Modelagem de banco relacional por meio do structureId
- Construir o processamento individual de cada tabela
- Cada tabela deve partir da mesma origem do dado

In [1]:
import os
from pathlib import Path

import pandas as pd

In [2]:
rootPath = os.path.abspath('..')
dataPath = os.path.join(rootPath, 'data')
trusted_path = Path(f"{dataPath}/trusted")
refined_path = Path(f"{dataPath}/refined")

In [3]:
df_sequence = pd.read_parquet(os.path.join(trusted_path, 'data_sequence.parquet'))
df_structure = pd.read_parquet(os.path.join(trusted_path, 'data_structure.parquet'))

In [4]:
# mapeando dos dados estruturais quais colunas ficam 1:1 com Id e quais são 1:*
df_structure.query("STRUCTUREID_STR == '2FYM'")

,STRUCTUREID_STR,CLASSIFICATION_STR,EXPERIMENTALTECHNIQUE_STR,MACROMOLECULETYPE_STR,RESIDUECOUNT_INT,RESOLUTION_FLOAT,STRUCTUREMOLECULARWEIGHT_FLOAT,CRYSTALLIZATIONMETHOD_STR,CRYSTALLIZATIONTEMPK_FLOAT,DENSITYMATTHEWS_FLOAT,DENSITYPERCENTSOL_FLOAT,PDBXDETAILS_STR,PHVALUE_FLOAT,PUBLICATIONYEAR_INT
38806,2FYM,LYASE,X-RAY DIFFRACTION,Protein,1760,1.6,186502.5,"VAPOR DIFFUSION, HANGING DROP",298.0,NaN,NaN,"8% PEG 4K, 0.2 M imidazole maleate, pH 6.0, VA...",6.0,2006.0
38807,2FYM,LYASE,X-RAY DIFFRACTION,Protein,1760,1.6,186502.5,"VAPOR DIFFUSION, HANGING DROP",298.0,NaN,NaN,"27% PEG 600, 0.1M HEPES, pH 7.5, VAPOR DIFFUSI...",7.5,2006.0
38808,2FYM,LYASE,X-RAY DIFFRACTION,Protein,1760,1.6,186502.5,"VAPOR DIFFUSION, HANGING DROP",298.0,2.47,50.11,"2.4M Sodium malonate, pH 7.0, VAPOR DIFFUSION,...",7.0,2006.0
38809,2FYM,LYASE,X-RAY DIFFRACTION,Protein,1760,1.6,186502.5,"VAPOR DIFFUSION, HANGING DROP",298.0,NaN,NaN,"40% MPEG 550, 0.1M HEPES, pH 8.2, VAPOR DIFFUS...",8.2,2006.0


In [5]:
structure_single_info = [
    'STRUCTUREID_STR',
    'CLASSIFICATION_STR',
    'EXPERIMENTALTECHNIQUE_STR',
    'MACROMOLECULETYPE_STR',
    'RESIDUECOUNT_INT',
    'RESOLUTION_FLOAT',
    'STRUCTUREMOLECULARWEIGHT_FLOAT',
    'CRYSTALLIZATIONMETHOD_STR',
    'CRYSTALLIZATIONTEMPK_FLOAT'
]

structure_multiple_info = [
    'DENSITYMATTHEWS_FLOAT',
    'DENSITYPERCENTSOL_FLOAT',
    'PHVALUE_FLOAT',
    'PDBXDETAILS_STR'
]

In [6]:
# mapeando da base com as sequencias, possíveis infos repetidas já estraidas na fonte antiga

df_sequence.query("STRUCTUREID_STR == '2FYM'")

,STRUCTUREID_STR,CHAINID_STR,SEQUENCE_STR,RESIDUECOUNT_INT,MACROMOLECULETYPE_STR
96122,2FYM,A,SKIVKIIGREIIDSRGNPTVEAEVHLEGGFVGMAAAPSGASTGSRE...,1760,Protein
96123,2FYM,B,ASPELASGKVWIRYPIVR,1760,Protein
96124,2FYM,C,SKIVKIIGREIIDSRGNPTVEAEVHLEGGFVGMAAAPSGASTGSRE...,1760,Protein
96125,2FYM,D,SKIVKIIGREIIDSRGNPTVEAEVHLEGGFVGMAAAPSGASTGSRE...,1760,Protein
96126,2FYM,E,ASPELASGKVWIRYPIVR,1760,Protein
96127,2FYM,F,SKIVKIIGREIIDSRGNPTVEAEVHLEGGFVGMAAAPSGASTGSRE...,1760,Protein


In [7]:
sequence_info = [
    'STRUCTUREID_STR',
    'SEQUENCE_STR'
]

### Construindo as tabelas e o esquema lógico do banco

In [8]:
# agrupando colunas por contexto

# tabela contendo apenas a sequencia de aminoacidos
structure_sequence = [
    'STRUCTUREID_STR',
    'SEQUENCE_STR'
]

# tabela com as informacoes estruturais da molecula
structure_info = [
    'STRUCTUREID_STR',
    'RESIDUECOUNT_INT',
    'RESOLUTION_FLOAT',
    'STRUCTUREMOLECULARWEIGHT_FLOAT',
    'CRYSTALLIZATIONTEMPK_FLOAT',
    'CRYSTALLIZATIONMETHOD_STR'
]

# tabela com dados de analises fisico quimicas
structure_analitycs = [
    'STRUCTUREID_STR',
    'DENSITYMATTHEWS_FLOAT',
    'DENSITYPERCENTSOL_FLOAT',
    'PHVALUE_FLOAT'
]

# tabela com os dados de classificacao, tipo da estrutura
structure_type = [
    'STRUCTUREID_STR',
    'CLASSIFICATION_STR',
    'MACROMOLECULETYPE_STR',
    'EXPERIMENTALTECHNIQUE_STR'
]

# tabela com dados e detalhes de campos abertos
structure_open_details = [
    'STRUCTUREID_STR',
    'PDBXDETAILS_STR'
]

In [9]:
# funcao que constroi a tabela
def create_refined_table(df: pd.DataFrame, colums: list, id_single=False):
    df = df[colums] # filtra as colunas selecionadas
    df = df.dropna() # retira valores na
    if id_single == True: # caso não seja 1:* retira IDs duplicados
        df.drop_duplicates(subset='STRUCTUREID_STR')
    # seleciona colunas str e coloca todas maiusculas
    colunas_str = [col for col in df.columns if col.endswith('STR')]
    df[colunas_str] = df[colunas_str].apply(lambda x: x.str.upper())
    return df

def save_parquet(df: pd.DataFrame, folder, file_name):
    path_save = os.path.join(folder, file_name)
    return df.to_parquet(path_save, index=False)

### Processamento das tabelas

In [10]:
config = {
    'sequencia_estrutura' : {
        'origen_file' : 'data_sequence.parquet',
        'columns_selected' : structure_sequence  
    },
    'infos_estrutura' : {
        'origen_file' : 'data_structure.parquet',
        'columns_selected' : structure_info  
    },
    'valores_analiticos' : {
        'origen_file' : 'data_structure.parquet',
        'columns_selected' : structure_analitycs  
    },
    'tipo_estrutura' : {
        'origen_file' : 'data_structure.parquet',
        'columns_selected' : structure_type  
    },
    'detalhes_gerais' : {
        'origen_file' : 'data_structure.parquet',
        'columns_selected' : structure_type  
    }
}

In [11]:
for table in config:
    df = pd.read_parquet(os.path.join(trusted_path, config[table]['origen_file']))
    columns_selected = config[table]['columns_selected']
    save_parquet(
        df=create_refined_table(df, columns_selected),
        folder=refined_path, 
        file_name=f'{table}.parquet'
    )

#### Esquema lógico do banco

![Arquitetura](../docs/img/database.png)